# Data Wrangling Project
## Cleaning a Job Postings Data Set 
#### Scott Lee


Date: 14/06/2021

Version: 1.0

Environment: Python 3.8.3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas 
* re 
* numpy
* datetime
* difflib
* Collections



## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

from datetime import datetime

df = pd.read_csv('dataset1_with_error.csv')


Taking a quick look at the dataframe so we know what we need to clean. 


In [2]:
df

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
0,12612628,Engineering Systems Analyst,Dorking,Gregory Martin International,NaN,permanent,Engineering Jobs,25000,20130708T120000,20130906T120000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Glasgow,Gregory Martin International,NaN,permanent,Engineering Jobs,30000,20120130T000000,20120330T000000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Hampshire,Gregory Martin International,NaN,permanent,Engineering Jobs,30000,20121221T150000,20130120T150000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Surrey,Gregory Martin International,NaN,permanent,Engineering Jobs,27500,20131208T150000,20140206T150000,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst",Surrey,Gregory Martin International,NaN,permanent,Engineering Jobs,25000,20130302T120000,20130501T120000,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
55164,72705203,TEACHER OF BUSINESS STUDIES AND LAW,Salisbury,,,contract,Teaching Jobs,22800,20120123T120000,20120206T120000,hays.co.uk
55165,72705205,Pensions Administrators (Temporary/Contract),UK,Abenefit2u,NaN,contract,Accounting & Finance Jobs,24000,20130801T150000,20130831T150000,cv-library.co.uk
55166,72705221,Senior Financial Advisor,London,Fram Executive Search.,-,permanent,Accounting & Finance Jobs,40000,20130126T000000,20130225T000000,ifaonlinejobs.co.uk
55167,72705240,Barclays Future Leaders Development Programmes,Hackney,Barclays,-,-,IT Jobs,36000,20121223T150000,20130221T150000,grb.uk.com


There are a couple of observations here:
<br> If something like a company name or job description is spelt wrong that will impact the analysis. 
<br> The date formats are not easily readable.
<br> There are missing values.

## Task 1: Auditing and cleansing the loaded data
<br>
Here we are going to have a look at the low frequency company names. This can help to give us an idea if a name may have been spelt wrong. 

In [3]:
#check the similar between two string
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

from collections import Counter
    
# find the high frequency location to low frequency location
def match_highfreq_To_lowfreq(data, min_threshold, max_threshold):
    # check low frequency company names
    lowfreq_data_list = [k for k,v in Counter(data).items() if v<=5]
    print ("Number of lowfreq_data", len(lowfreq_data_list))
    highfreq_data_list = list(set(data[~(data.isnull())]).difference(lowfreq_data_list))
    print ("Number of highfreq_data", len(highfreq_data_list))
    # the printed data tells that probably not many companies posted more than 5 jobs in those years.


    match_data={}
    for lowfreq_data in lowfreq_data_list:
        for highfreq_data in highfreq_data_list:
            ratio = similar(lowfreq_data.upper(), highfreq_data.upper())
            if ratio >= min_threshold and ratio <= max_threshold:
                match_data[lowfreq_data] = highfreq_data
                print(lowfreq_data, ":", highfreq_data, ratio)
    return match_data

## Id column
Some observations before we begin:
<br> Are all the records the same data type and length?
<br> Are they all unique? 
<br> We will investigate below. 

In [4]:
type(df.iloc[1,0])
#DF.id is an integer

#Handling the Id column. Confirming data type is int and that all Id's are 8 digits in length

test = df

holding_list = []
for item in test.Id:
    item = str(item)
    if len(item) == 8:
        pass
    else:
        print('Fail')
        holding_list.append(item)
if len(holding_list)>0:
    print('Warning, there is an error')
else:
    print("All Id's are integer of length 8")

df.Id.astype(int)

print('The number of unique id entries are', len(df.Id.unique()))
print('This is equal to the number of rows, thus all are unique')

All Id's are integer of length 8
The number of unique id entries are 55169
This is equal to the number of rows, thus all are unique


## Title Column
<br> Their are job titles that semantically mean the same thing but are written differently. E.g. "software engineer" and "Software Engineer", 
<br> Also there are location names that appear in a job title, e.g. "Analyst - London", needs to be "Analyst" with "London moved to a different field. 

In [5]:
#Cleaning up and normalising the title data
def clean_title(job_title):
    if pd.isnull(job_title):
        return job_title
    else:
        #Normalise to uppercase letters
        job_title = job_title.upper()
        #Remove special characters at the end
        job_title = re.sub(r"\W+$", "", job_title) 
        #Remove all special charactrs except space and dot
        #Included "+" charcter and "/" charcter as they are relevant e.g. a C++ programer job loses meaning without "++"
        job_title = re.sub(r'([^\w\s\.\+\/]|_)', '', job_title)
        #Replace multiple spaces with a single space aksi turn spaces on both sides
        job_title = re.sub('\s+', ' ', job_title).strip()
        return job_title

    
print(len(df.Title))
df['TitleNormalised'] = df.Title.apply(lambda x: clean_title(x))
print(len(set(df.TitleNormalised)), 'different job titles after normalisation')


#Remove location names if they appear in the job title column, i.e. redundent data.

for i in range(0, len(df.Id)):
    title_desc = df.iloc[i,11]
    title_loc = df.iloc[i,2].upper()
    if df.iloc[i,2].upper() in df.iloc[i,11]:
        title_desc_new = title_desc.replace(title_loc, '')
        df.iloc[i,11] = title_desc_new
    else:
        title_desc_new = title_desc
        df.iloc[i,11] = title_desc_new
        
#Drop Title column as correct data is migrated to 'TitleNormalised'
df.drop('Title', axis = 1, inplace = True)


55169
55165 different job titles after normalisation


Below we will use some NLP techniques to check for job names that essentially match. If the job name mateches to a 95% threshold, we will merge the low frequency instances with the high frequency incidents. 

In [6]:
match_data=match_highfreq_To_lowfreq(df.TitleNormalised, 0.95, 1)

Number of lowfreq_data 52447
Number of highfreq_data 117
BUSINESS DEVELOPMENT MANAGER : BUSINESS DEVELOPMENT MANAGER  0.9824561403508771
CARE ASSISTANTS  : CARE ASSISTANT  0.967741935483871
STAFF NURSE RGN  K : STAFF NURSE RGN   0.9714285714285714
SUPPORT WORKER : SUPPORT WORKER  0.9655172413793104
STAFF NURSE RGN : STAFF NURSE RGN  0.967741935483871
NIGHT NURSE : NIGHT NURSE  0.9565217391304348
CARE HOME MANAGER : CARE HOME MANAGER  0.9714285714285714
HOME MANAGER : HOME MANAGER  0.96
DEPUTY NURSING HOME MANAGER : DEPUTY NURSING HOME MANAGER  0.9818181818181818
STAFF NURSE : STAFF NURSE  0.9565217391304348
CARE ASSISTANT : CARE ASSISTANT  0.9655172413793104
OCCUPATIONAL THERAPIST : OCCUPATIONAL THERAPIST  0.9777777777777777
REGISTERED NURSE  RGN : REGISTERED NURSE RGN  0.9523809523809523
REGISTERED MANAGER : REGISTERED MANAGER  0.972972972972973
STAFF NURSES  : STAFF NURSE  0.96
NURSING HOME MANAGER : NURSING HOME MANAGER  0.975609756097561
DEPUTY MANAGER : DEPUTY MANAGER  0.965517241

DEPUTY NURSING HOME MANAGERS  : DEPUTY NURSING HOME MANAGER  0.9824561403508771
DEPUTY MANAGER ELDERLY NURSING HOME  : DEPUTY MANAGER  ELDERLY NURSING HOME 0.9722222222222222
RESTAURANT MANAGER K : RESTAURANT MANAGER  0.9743589743589743
ASSISTANT RESTAURANT MANAGER  K : ASSISTANT RESTAURANT MANAGER  0.9666666666666667
RESGISTERED GENERAL NURSE : REGISTERED GENERAL NURSE  0.96
BUSINESS ANALYST X : BUSINESS ANALYST  0.9714285714285714
BUSINESS DEVELOPEMNT MANAGER  : BUSINESS DEVELOPMENT MANAGER  0.9655172413793104
UNIT MANAGER   : UNIT MANAGER  0.9629629629629629
STAFF NURSE NURSING HOME   : STAFF NURSE NURSING HOME  0.9803921568627451
DEPUTY HOME MANAGER RN  : DEPUTY HOME MANAGER RGN  0.9787234042553191
REGISTERED GENERAL NURSE / RGN  : REGISTERED GENERAL NURSE RGN  0.9666666666666667
DEPUTY MANAGER RGN  P : DEPUTY MANAGER RGN  0.95
PHLEBOTOMIST : PHLEBOTOMIST  0.96
NIGHT NURSES  : NIGHT NURSE  0.96
CARE HOME MANAGER   : CARE HOME MANAGER  0.972972972972973
BUSINESS ANALYST T : BUSINESS

In [7]:
df['job_title_resovle_similar'] = df.TitleNormalised.replace(match_data)

In [8]:
df.drop('TitleNormalised', axis = 1, inplace = True)
df['Title'] = df.job_title_resovle_similar.replace(match_data)
df.drop('job_title_resovle_similar', axis = 1, inplace = True)

## Location

In [9]:
def clean_location(location):
    if pd.isnull(location):
        return location
    else:
        #Normalise to uppercase letters
        location = location.upper()
        #Remove special characters at the end
        location = re.sub(r"\W+$", "", location) 
        #Remove all special charactrs except space and dot
        #Included "+" charcter and "/" charcter as they are relevant e.g. a C++ programer job loses meaning without "++"
        location = re.sub(r'([^\w\s\.\+\/]|_)', '', location)
        #Replace multiple spaces with a single space aksi turn spaces on both sides
        location = re.sub('\s+', ' ', location).strip()
        return location

    
print(len(df.Location))
df['LocationNormalised'] = df.Location.apply(lambda x: clean_location(x))
print(len(set(df.LocationNormalised)), 'different locations after normalisation')

df.drop('Location', axis = 1, inplace = True)

print('Location cloumn is datatype ', type(df.loc[1,'LocationNormalised']))

55169
484 different locations after normalisation
Location cloumn is datatype  <class 'str'>


## Company
<br> Conducting a similar analysis on the Company column.

In [10]:
def clean_company(company):
    if pd.isnull(company):
        return company
    else:
        #Normalise to uppercase letters
        company = company.upper()
        #Remove special characters at the end
        company = re.sub(r"\W+$", "", company) 
        #Remove all special charactrs except space and dot
        #Included "+" charcter and "/" charcter as they are relevant e.g. a C++ programer job loses meaning without "++"
        company = re.sub(r'([^\w\s\.\+\/]|_)', '', company)
        #Replace multiple spaces with a single space aksi turn spaces on both sides
        company = re.sub('\s+', ' ', company).strip()
        return company
    
print(len(df.Company))
df['CompanyNormalised'] = df.Company.apply(lambda x: clean_company(x))
print(len(set(df.CompanyNormalised)), 'different companies after normalisation')

df.drop('Company', axis = 1, inplace = True)

55169
8671 different companies after normalisation


In [11]:
df.CompanyNormalised.value_counts()

                           1540
UKSTAFFSEARCH               498
RANDSTAD                    403
CVBROWSER                   328
MATCHTECH GROUP PLC         286
                           ... 
TUI UK IRELAND                1
SEARCH 4 TECHNOLOGY LTD       1
SILVERBUG LTD                 1
SPRING GROUP BRISTON          1
ADRENALIN LTD                 1
Name: CompanyNormalised, Length: 8670, dtype: int64

In [12]:
match_data=match_highfreq_To_lowfreq(df.CompanyNormalised, 0.95, 1)

Number of lowfreq_data 6729
Number of highfreq_data 1941
EDSTAFF LTD : EDUSTAFF LTD 0.9565217391304348
BMS GRADUATE : BMS GRADUATES 0.96
SOLUTION RECRUITMENT : SOLUTIONS RECRUITMENT 0.975609756097561
SIGMAR RECRUITMENT : SIGMA RECRUITMENT 0.9714285714285714
TRAVAIL EMPLYMENT GROUP : TRAVAIL EMPLOYMENT GROUP 0.9787234042553191
SHAW HEALTH CARE : SHAW HEALTHCARE 0.967741935483871
DERBYSHIRE COUNTY COUNCIL 2 : DERBYSHIRE COUNTY COUNCIL 0.9615384615384616
TRAVAIL EMPLOYMNET GROUP : TRAVAIL EMPLOYMENT GROUP 0.9583333333333334
ASHTON RECRUITMENT : ASTON RECRUITMENT 0.9714285714285714
SOLUTIONS2RECRUITMENT : SOLUTIONS RECRUITMENT 0.9523809523809523
BADENOCHCLARK : BADENOCH CLARK 0.9629629629629629
ADDITIONAL RESOURCE : ADDITIONAL RESOURCES 0.9743589743589743
ANGLIAN HOME IMPROVEMENT : ANGLIAN HOME IMPROVEMENTS 0.9795918367346939
ONE RECRUITMENT : NE RECRUITMENT 0.9655172413793104
EASY WEB RECRUITMENT : EASYWEB RECRUITMENT 0.9743589743589743
PRIMETIME RECRUITMENT : PRIME TIME RECRUITMENT 0.976

In [13]:
# replace lower frequent with higher frequent company name
df['Company_resovle_similar'] = df.CompanyNormalised.replace(match_data)


In [14]:
df['Company'] = df['Company_resovle_similar']
df.drop('Company_resovle_similar', axis = 1, inplace = True)
df.drop('CompanyNormalised', axis = 1, inplace = True)

In [15]:
len(df.Company.unique())

8622

## Contract Type
<br> Contract type has some missing data. There is no accurate way of inferring a contract type based off of the other data, so this will need to be dealt with. 

In [16]:
print('The unique values of ContractType are' ,df.ContractType.unique())
print('\n')
print('The value counts of ContractType are: \n', df.ContractType.value_counts())

The unique values of ContractType are [nan '-' ' ' 'full_time' 'part_time']


The value counts of ContractType are: 
              14902
full_time    12303
-             4720
part_time     1568
Name: ContractType, dtype: int64


There is no way to infer the contract type from the other data so anything that is not labelled will be changed to " Contract Type Not Specified" 
<br>
Also data types of this column are mixed as both float and strings, so convert all to type string for processing

In [17]:
#Changing the nan and empty values to a single string value of 'non_specified'
df['ContractType'] = df.ContractType.astype(str)
for i in range(0,len(df['ContractType'])):
    if df.loc[i,'ContractType'] == 'nan':
        df.loc[i,'ContractType'] = 'non_specified'
    elif df.loc[i,'ContractType'] == '-':
        df.loc[i,'ContractType'] = 'non_specified'
    elif df.loc[i,'ContractType'] == ' ':
        df.loc[i,'ContractType'] = 'non_specified'     
    else:
        pass

print('The unique ContractTypes after wrangling:', df.ContractType.unique())

The unique ContractTypes after wrangling: ['non_specified' 'full_time' 'part_time']


## Contract_Time

In [18]:
print('The unique ContractTime values prior to wrangling are: \n', df.ContractTime.unique())

#Amend to nan, '-' and ' ' to a single value of 'non-specified'
df['ContractTime'] = df.ContractTime.astype(str)

for i in range(0,len(df.ContractTime)):
    if df.loc[i,'ContractTime'] == 'nan':
        df.loc[i,'ContractTime'] = 'non-specified'
    if df.loc[i,'ContractTime'] == '-':
        df.loc[i,'ContractTime'] = 'non-specified'
    elif df.loc[i,'ContractTime'] == ' ':
        df.loc[i,'ContractTime'] = 'non-specified'
    else:
        pass

print('The unique values after wrangling are: \n', df.ContractTime.unique())

The unique ContractTime values prior to wrangling are: 
 ['permanent' '-' 'contract' nan ' ']
The unique values after wrangling are: 
 ['permanent' 'non-specified' 'contract']


## Category

In [19]:
print('The number of unique values: \n', len(df.Category.unique()))
print('\n')
print(df.Category.value_counts())

#Confirm all values a re type string
df['Category'] = df.Category.astype(str)

The number of unique values: 
 8


IT Jobs                             14344
Healthcare & Nursing Jobs            8809
Engineering Jobs                     8210
Accounting & Finance Jobs            7138
Sales Jobs                           5349
Hospitality & Catering Jobs          4788
Teaching Jobs                        3779
PR, Advertising & Marketing Jobs     2752
Name: Category, dtype: int64


There are only 8 categories, all of which are strings.
<br> 
Ok to leave as is.

## Open Date and Close Date
<br> The date format is very inconsistent, will use regular expressions to clean this up. 

In [20]:
#Function to apply regex to open_date and filter out inconsistancies
def clean_open_date(open_date):
    if pd.isnull(open_date):
        return open_date
    
    else:
        open_date = re.sub(r'\d{4}1[3-9]\d{2}T\d+',open_date[0:4] + open_date[6:8] + open_date[4:6] + open_date[8:],open_date)
        open_date = re.sub(r'\d{4}2[0-9]\d{2}T\d+',open_date[0:4] + open_date[6:8] + open_date[4:6] + open_date[8:],open_date)
    return open_date

df['OpenDate'] = df.OpenDate.apply(lambda x: clean_open_date(x))


#For loop to change to the desired format
for i in range(0, len(df.OpenDate)):  
    x = df.loc[i,'OpenDate']
    date = x.split('T')[0]
    time_comp = x.split('T')[1]
    time_comp_new = time_comp[0:2] + ':' + time_comp[2:4] + ':' + time_comp[4:6]
    time_comp_new

    #Remormated date to read into datetime module
    date_read_in = date + " " + time_comp_new

    #Date object
    date_object = datetime.strptime(date_read_in,'%Y%m%d %H:%M:%S')
    df.loc[i,'OpenTimeNormalised'] = date_object

In [21]:
#Close Date Module
for i in range(0, len(df.CloseDate)):  
    x = df.loc[i,'CloseDate']
    date = x.split('T')[0]
    time_comp = x.split('T')[1]
    time_comp_new = time_comp[0:2] + ':' + time_comp[2:4] + ':' + time_comp[4:6]
    time_comp_new

    #Remormated date to read into datetime module
    date_read_in = date + " " + time_comp_new

    #Date object
    date_object = datetime.strptime(date_read_in,'%Y%m%d %H:%M:%S')
    df.loc[i,'CloseTimeNormalised'] = date_object

In [22]:
#Drop original open and close date columns
df.drop('OpenDate', axis = 1, inplace = True)
df.drop('CloseDate', axis = 1, inplace = True)

## SourceLink
<br>
Checking all values are strings, value counts and number of uniques.
<br> 
Also converting to uppercase and comparing to check for duplicates, then dropping capitals

In [23]:
#Exploratory analysis:
len_source = len(df.Source.unique())
print('The number of unique source names is: ',len_source)
print('Value count of df.Source: \n', df.Source.value_counts())

count = 0
for i in range(0, len(df.Source)):
    if type(df.loc[i, 'Source']) is str:
        count = count + 1
    else:
        pass
print('The number of values in the data frame that are type string =', count)


count = 0
for i in range(0, len(df.Source)):
    if type(df.loc[i,'Source']) is not str:
        count = count + 1
    else:
        pass
print('The number of values in the data frame that are not type string =', count)

The number of unique source names is:  106
Value count of df.Source: 
 totaljobs.com        10102
cv-library.co.uk      7841
jobsite.co.uk         3630
cwjobs.co.uk          3132
staffnurse.com        2778
                     ...  
workthing.com            5
scotsman.com             4
jobs.gponline.com        3
cvjobstore.com           1
thegraduate.co.uk        1
Name: Source, Length: 106, dtype: int64
The number of values in the data frame that are type string = 55169
The number of values in the data frame that are not type string = 0


In [24]:
#Converting to uppercase then comparing to see if any difference. 
#Will then drop upper case column ('Source_Normalised)

for i in range(0,len(df.Source)):
    df.loc[i,'Source'].upper()
    df.loc[i,'Source_Normalised'] = df.loc[i,'Source'].upper()

#Exploritory Analysis
len_source = len(df.Source_Normalised.unique())
print('The number of unique source names is: ',len_source)
print('Value count of df.Source: \n', df.Source_Normalised.value_counts())

count = 0
for i in range(0, len(df.Source_Normalised)):
    if type(df.loc[i, 'Source_Normalised']) is str:
        count = count + 1
    else:
        pass
print('The number of values in the data frame that are type string =', count)


count = 0
for i in range(0, len(df.Source_Normalised)):
    if type(df.loc[i,'Source_Normalised']) is not str:
        count = count + 1
    else:
        pass
print('The number of values in the data frame that are not type string =', count)


The number of unique source names is:  106
Value count of df.Source: 
 TOTALJOBS.COM        10102
CV-LIBRARY.CO.UK      7841
JOBSITE.CO.UK         3630
CWJOBS.CO.UK          3132
STAFFNURSE.COM        2778
                     ...  
GRB.UK.COM               5
SCOTSMAN.COM             4
JOBS.GPONLINE.COM        3
THEGRADUATE.CO.UK        1
CVJOBSTORE.COM           1
Name: Source_Normalised, Length: 106, dtype: int64
The number of values in the data frame that are type string = 55169
The number of values in the data frame that are not type string = 0


In [25]:
match_data=match_highfreq_To_lowfreq(df.Source_Normalised,0.7,1)

Number of lowfreq_data 6
Number of highfreq_data 100
CVJOBSTORE.COM : JOBSERVE.COM 0.7692307692307693


Based on the above, I'm satisfied there are is no signiificant level or mis-spellings or typos of names requiring fixing. 

In [26]:
#Drop new column 'Source_Normalised' after analysis
df.drop('Source_Normalised', axis = 1, inplace = True)

## Salary

In [27]:
def clean_salary(salary):
    if pd.isnull(salary):
        return salary
    else:       
        salary = re.sub(r"-","0", salary)
        salary = re.sub(r"nan", "0", salary)
        salary = re.split(r"/",salary)[0]
        salary = re.split(r"\s", salary)[0]
        #salary = re.split(r"[k|K]", salary[0:2])
        salary = re.sub(r"\d+[k|K]", salary[0:2] + '000', salary)
        
        return salary

df['Salary_Normalised'] = df.Salary.apply(lambda x: clean_salary(x))

df.Salary_Normalised = df.Salary_Normalised.astype(float)

In [28]:
df.drop('Salary', axis = 1, inplace = True)

In [29]:
df

,Id,ContractType,ContractTime,Category,Source,Title,LocationNormalised,Company,OpenTimeNormalised,CloseTimeNormalised,Salary_Normalised
0,12612628,non_specified,permanent,Engineering Jobs,cv-library.co.uk,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,2013-07-08 12:00:00,2013-09-06 12:00:00,25000.0
1,12612830,non_specified,permanent,Engineering Jobs,cv-library.co.uk,STRESS ENGINEER,GLASGOW,GREGORY MARTIN INTERNATIONAL,2012-01-30 00:00:00,2012-03-30 00:00:00,30000.0
2,12612844,non_specified,permanent,Engineering Jobs,cv-library.co.uk,MODELLING AND SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,2012-12-21 15:00:00,2013-01-20 15:00:00,30000.0
3,12613049,non_specified,permanent,Engineering Jobs,cv-library.co.uk,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,SURREY,GREGORY MARTIN INTERNATIONAL,2013-12-08 15:00:00,2014-02-06 15:00:00,27500.0
4,12613647,non_specified,permanent,Engineering Jobs,cv-library.co.uk,PIONEER MISER ENGINEERING SYSTEMS ANALYST,SURREY,GREGORY MARTIN INTERNATIONAL,2013-03-02 12:00:00,2013-05-01 12:00:00,25000.0
...,...,...,...,...,...,...,...,...,...,...,...
55164,72705203,non_specified,contract,Teaching Jobs,hays.co.uk,TEACHER OF BUSINESS STUDIES AND LAW,SALISBURY,,2012-01-23 12:00:00,2012-02-06 12:00:00,22800.0
55165,72705205,non_specified,contract,Accounting & Finance Jobs,cv-library.co.uk,PENSIONS ADMINISTRATORS TEMPORARY/CONTRACT,UK,ABENEFIT2U,2013-08-01 15:00:00,2013-08-31 15:00:00,24000.0
55166,72705221,non_specified,permanent,Accounting & Finance Jobs,ifaonlinejobs.co.uk,SENIOR FINANCIAL ADVISOR,LONDON,FRAM EXECUTIVE SEARCH,2013-01-26 00:00:00,2013-02-25 00:00:00,40000.0
55167,72705240,non_specified,non-specified,IT Jobs,grb.uk.com,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,HACKNEY,BARCLAYS,2012-12-23 15:00:00,2013-02-21 15:00:00,36000.0


## Format Final DataFrame

In [30]:
df.head()

,Id,ContractType,ContractTime,Category,Source,Title,LocationNormalised,Company,OpenTimeNormalised,CloseTimeNormalised,Salary_Normalised
0,12612628,non_specified,permanent,Engineering Jobs,cv-library.co.uk,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,2013-07-08 12:00:00,2013-09-06 12:00:00,25000.0
1,12612830,non_specified,permanent,Engineering Jobs,cv-library.co.uk,STRESS ENGINEER,GLASGOW,GREGORY MARTIN INTERNATIONAL,2012-01-30 00:00:00,2012-03-30 00:00:00,30000.0
2,12612844,non_specified,permanent,Engineering Jobs,cv-library.co.uk,MODELLING AND SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,2012-12-21 15:00:00,2013-01-20 15:00:00,30000.0
3,12613049,non_specified,permanent,Engineering Jobs,cv-library.co.uk,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,SURREY,GREGORY MARTIN INTERNATIONAL,2013-12-08 15:00:00,2014-02-06 15:00:00,27500.0
4,12613647,non_specified,permanent,Engineering Jobs,cv-library.co.uk,PIONEER MISER ENGINEERING SYSTEMS ANALYST,SURREY,GREGORY MARTIN INTERNATIONAL,2013-03-02 12:00:00,2013-05-01 12:00:00,25000.0


From the above:
- Source to become SourceName
- LocationNormalised to become Location
- OpenTimeNormalised to become OpenDate
- CloseTimeNormalised to become CloseDate
- Salary Normalised to become Salary

In [31]:
df['SourceName'] = df['Source']
df.drop('Source', axis = 1, inplace = True)

df['Location'] = df['LocationNormalised']
df.drop('LocationNormalised', axis = 1, inplace = True)

df['OpenDate'] = df['OpenTimeNormalised']
df.drop('OpenTimeNormalised', axis = 1, inplace = True)

df['CloseDate'] = df['CloseTimeNormalised']
df.drop('CloseTimeNormalised', axis = 1, inplace = True)

df['Salary'] = df['Salary_Normalised']
df.drop('Salary_Normalised', axis = 1, inplace = True)

In [32]:
df.head()

,Id,ContractType,ContractTime,Category,Title,Company,SourceName,Location,OpenDate,CloseDate,Salary
0,12612628,non_specified,permanent,Engineering Jobs,ENGINEERING SYSTEMS ANALYST,GREGORY MARTIN INTERNATIONAL,cv-library.co.uk,DORKING,2013-07-08 12:00:00,2013-09-06 12:00:00,25000.0
1,12612830,non_specified,permanent,Engineering Jobs,STRESS ENGINEER,GREGORY MARTIN INTERNATIONAL,cv-library.co.uk,GLASGOW,2012-01-30 00:00:00,2012-03-30 00:00:00,30000.0
2,12612844,non_specified,permanent,Engineering Jobs,MODELLING AND SIMULATION ANALYST,GREGORY MARTIN INTERNATIONAL,cv-library.co.uk,HAMPSHIRE,2012-12-21 15:00:00,2013-01-20 15:00:00,30000.0
3,12613049,non_specified,permanent,Engineering Jobs,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,GREGORY MARTIN INTERNATIONAL,cv-library.co.uk,SURREY,2013-12-08 15:00:00,2014-02-06 15:00:00,27500.0
4,12613647,non_specified,permanent,Engineering Jobs,PIONEER MISER ENGINEERING SYSTEMS ANALYST,GREGORY MARTIN INTERNATIONAL,cv-library.co.uk,SURREY,2013-03-02 12:00:00,2013-05-01 12:00:00,25000.0


In [33]:
#Rearrange columns to ensure desired format for output

In [34]:
df = df[['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime', 'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName']]

In [35]:
df

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,12612628,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,25000.0,2013-07-08 12:00:00,2013-09-06 12:00:00,cv-library.co.uk
1,12612830,STRESS ENGINEER,GLASGOW,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,30000.0,2012-01-30 00:00:00,2012-03-30 00:00:00,cv-library.co.uk
2,12612844,MODELLING AND SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,30000.0,2012-12-21 15:00:00,2013-01-20 15:00:00,cv-library.co.uk
3,12613049,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,SURREY,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,27500.0,2013-12-08 15:00:00,2014-02-06 15:00:00,cv-library.co.uk
4,12613647,PIONEER MISER ENGINEERING SYSTEMS ANALYST,SURREY,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,25000.0,2013-03-02 12:00:00,2013-05-01 12:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
55164,72705203,TEACHER OF BUSINESS STUDIES AND LAW,SALISBURY,,non_specified,contract,Teaching Jobs,22800.0,2012-01-23 12:00:00,2012-02-06 12:00:00,hays.co.uk
55165,72705205,PENSIONS ADMINISTRATORS TEMPORARY/CONTRACT,UK,ABENEFIT2U,non_specified,contract,Accounting & Finance Jobs,24000.0,2013-08-01 15:00:00,2013-08-31 15:00:00,cv-library.co.uk
55166,72705221,SENIOR FINANCIAL ADVISOR,LONDON,FRAM EXECUTIVE SEARCH,non_specified,permanent,Accounting & Finance Jobs,40000.0,2013-01-26 00:00:00,2013-02-25 00:00:00,ifaonlinejobs.co.uk
55167,72705240,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,HACKNEY,BARCLAYS,non_specified,non-specified,IT Jobs,36000.0,2012-12-23 15:00:00,2013-02-21 15:00:00,grb.uk.com


# Final Data Frame
<br> Below is the celaned data frame, we will simply print it out to see what it looks like. 

In [36]:
df

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,12612628,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,25000.0,2013-07-08 12:00:00,2013-09-06 12:00:00,cv-library.co.uk
1,12612830,STRESS ENGINEER,GLASGOW,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,30000.0,2012-01-30 00:00:00,2012-03-30 00:00:00,cv-library.co.uk
2,12612844,MODELLING AND SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,30000.0,2012-12-21 15:00:00,2013-01-20 15:00:00,cv-library.co.uk
3,12613049,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,SURREY,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,27500.0,2013-12-08 15:00:00,2014-02-06 15:00:00,cv-library.co.uk
4,12613647,PIONEER MISER ENGINEERING SYSTEMS ANALYST,SURREY,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,25000.0,2013-03-02 12:00:00,2013-05-01 12:00:00,cv-library.co.uk
...,...,...,...,...,...,...,...,...,...,...,...
55164,72705203,TEACHER OF BUSINESS STUDIES AND LAW,SALISBURY,,non_specified,contract,Teaching Jobs,22800.0,2012-01-23 12:00:00,2012-02-06 12:00:00,hays.co.uk
55165,72705205,PENSIONS ADMINISTRATORS TEMPORARY/CONTRACT,UK,ABENEFIT2U,non_specified,contract,Accounting & Finance Jobs,24000.0,2013-08-01 15:00:00,2013-08-31 15:00:00,cv-library.co.uk
55166,72705221,SENIOR FINANCIAL ADVISOR,LONDON,FRAM EXECUTIVE SEARCH,non_specified,permanent,Accounting & Finance Jobs,40000.0,2013-01-26 00:00:00,2013-02-25 00:00:00,ifaonlinejobs.co.uk
55167,72705240,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,HACKNEY,BARCLAYS,non_specified,non-specified,IT Jobs,36000.0,2012-12-23 15:00:00,2013-02-21 15:00:00,grb.uk.com


## Saving Data

## Export to CSV

In [37]:
df.to_csv('dataset1_solution.csv', index = False)

In [38]:
#Check output is as expected
check_df = pd.read_csv('dataset1_solution.csv')
check_df.head()

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,12612628,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,25000.0,2013-07-08 12:00:00,2013-09-06 12:00:00,cv-library.co.uk
1,12612830,STRESS ENGINEER,GLASGOW,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,30000.0,2012-01-30 00:00:00,2012-03-30 00:00:00,cv-library.co.uk
2,12612844,MODELLING AND SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,30000.0,2012-12-21 15:00:00,2013-01-20 15:00:00,cv-library.co.uk
3,12613049,ENGINEERING SYSTEMS ANALYST / MATHEMATICAL MOD...,SURREY,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,27500.0,2013-12-08 15:00:00,2014-02-06 15:00:00,cv-library.co.uk
4,12613647,PIONEER MISER ENGINEERING SYSTEMS ANALYST,SURREY,GREGORY MARTIN INTERNATIONAL,non_specified,permanent,Engineering Jobs,25000.0,2013-03-02 12:00:00,2013-05-01 12:00:00,cv-library.co.uk


## Summary

The data set has been cleaned so that it is now able to be processed in a later notebook. Some things we did:
<br> Checked for semantically identical titles and decriptions by using NLP tools to check frequencies of items.
<br> Cleaned up different formatting of the same item, e.g. "Software Engineer" and "software engineer"
<br> Made date formated more easily readable
<br> Used our judgement to make certain assumptions and understand the limits of our data cleaning. E.g. if no contract type was specified, we listed it as such and couldn't infer that it was something else. 
